In [1]:
import pandas as pd
import numpy as np
from time import time
import re
import string
import socket

In [2]:
country_code = "US"
print('Country:', country_code)

# Local
if 'samuel' in socket.gethostname().lower():
    path_to_data = os.path.join('../../data/classification',country_code)
# Cluster
else:
    path_to_data = os.path.join('/scratch/spf248/twitter/data/classification',country_code)
print('Path to data:',path_to_data)

Country: US
Path to data: ../../data/classification/US


# Training Set

In [3]:
xls = pd.ExcelFile(os.path.join(path_to_data,'tweets-labeled-sana.xlsx'))

In [4]:
tweets = pd.DataFrame()

question2label = {
'1a - DOES THIS TWEET INDICATE THAT THE USER RECENTLY LOST OR LEFT A JOB? [Answer: YES/NO] (Example: I lost my job today — I was fired earlier this week — I recently got laid off — I just quit my job)':'job_loss',
'2 - DOES THIS TWEET INDICATE THAT THE USER IS CURRENTLY UNEMPLOYED? [Answer: YES/NO] (Example: Now I am unemployed — I am currently not working)':'is_unemployed',
'3a - DOES THIS TWEET INDICATE THAT THE USER IS SEARCHING FOR A JOB? [Answer: YES/NO] (Example: I am looking for a job — I am searching for a new position — Anyone hiring?)':'job_search',
'4a - DOES THIS TWEET INDICATE THAT THE USER WAS RECENTLY HIRED? [Answer: YES/NO] (Example: I just found a job — I got hired today — I started working at my new position on Monday — Can’t wait to start my new job!)':'is_hired',
'5 - DOES THIS TWEET CONTAIN A JOB OFFER? [Answer: YES/NO] (Example: Looking for a new position? — Here is a job opportunity you might be interested in)':'job_offer',
}

for sheet_name in xls.sheet_names:
    
    data = pd.read_excel(xls,sheet_name=sheet_name)
    
    # Drop Auxiliary Questions
    data.drop([x for x in data if 'b - ' in x],1,inplace=True)
    
    data.rename(columns=question2label,inplace=True)
    data.rename(columns=lambda x:x.lower(),inplace=True)
    
    data.dropna(inplace=True)
    
    tweets = pd.concat([tweets,data])
    
tweets = tweets.replace('No',0).replace('Yes',1)
tweets.id = tweets.id.astype(str)
tweets.set_index('id',inplace=True)
print('# Tweets', tweets.shape[0])

# Tweets 1862


In [5]:
tweets.head()

,text,job_loss,is_unemployed,job_search,is_hired,job_offer
id,,,,,,
348940747036323840,“@xolysha_: so you have a kid and unemployed #...,0,0,0,0,0
430590411212476417,wow being unemployed is almost worse than bein...,0,1,0,0,0
481577981207662593,"come october, i'll either be studying medicine...",0,0,0,0,0
555815463335120896,@jadeemond_ oh haha cause code red at work mea...,0,0,0,0,0
116942703002001408,@eonline levi and palin bunch need to go away!...,0,0,0,0,0


In [6]:
tweets.mean()

job_loss         0.118153
is_unemployed    0.211600
job_search       0.136412
is_hired         0.114930
job_offer        0.138561
dtype: float64

In [7]:
train = tweets.sample(frac=0.7,random_state=0)
dev = tweets.drop(train.index)

print('# Training Examples:', train.shape[0])
print('# Test Examples:', dev.shape[0])

# Training Examples: 1303
# Test Examples: 559


In [8]:
train.to_csv(os.path.join(path_to_data,'train.tsv'),sep='\t')
dev.to_csv(os.path.join(path_to_data,'dev.tsv'),sep='\t')
with open(os.path.join(path_to_data,'labels'), 'w') as f:
    for label in list(question2label.values()):
        print(label)
        f.write("%s\n" % label)

job_loss
is_unemployed
job_search
is_hired
job_offer
